![alt text](../img/explore-space-using-python.svg "Over the moon")
# Plan a Moon mission by using Python pandas

In this module, you will:

* Create a clear representation of data from many sources.
* Use Python and pandas to explore data.
* Use data cleansing techniques to get a clear representation of data.
* Hypothesize how much rock sample astronauts might bring back on the Artemis missions.



## Reads Dataset

In [114]:
import pandas as pd
rock_samples = pd.read_csv('data/rocksamples.csv')

# head() shows the first five lines of the DataFrame
rock_samples.head()

,ID,Mission,Type,Subtype,Weight(g),Pristine(%)
0,10001,Apollo11,Soil,Unsieved,125.8,88.36
1,10002,Apollo11,Soil,Unsieved,5629.0,93.73
2,10003,Apollo11,Basalt,Ilmenite,213.0,65.56
3,10004,Apollo11,Core,Unsieved,44.8,71.76
4,10005,Apollo11,Core,Unsieved,53.4,40.31


In [115]:

"""
    Describes the DataFrame
"""
rock_samples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2229 entries, 0 to 2228
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           2229 non-null   int64  
 1   Mission      2229 non-null   object 
 2   Type         2229 non-null   object 
 3   Subtype      2226 non-null   object 
 4   Weight(g)    2229 non-null   float64
 5   Pristine(%)  2229 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 104.6+ KB


## Convert Weight from grams to kilogram

In [116]:
rock_samples['Weight(g)'] = rock_samples['Weight(g)'].apply(lambda x : x * 0.001)
rock_samples.rename(columns={'Weight(g)' : 'Weight(Kg)'}, inplace=True)
rock_samples.head()

,ID,Mission,Type,Subtype,Weight(Kg),Pristine(%)
0,10001,Apollo11,Soil,Unsieved,0.1258,88.36
1,10002,Apollo11,Soil,Unsieved,5.6290,93.73
2,10003,Apollo11,Basalt,Ilmenite,0.2130,65.56
3,10004,Apollo11,Core,Unsieved,0.0448,71.76
4,10005,Apollo11,Core,Unsieved,0.0534,40.31


## Mission Dimension

The rock_samples dataframe has a row for every sample that was collected but, we want to understand the rock samples in total as they relate to the specific rockets that brought them back.

In [117]:
missions = pd.DataFrame()
missions['Mission'] = rock_samples['Mission'].unique()
missions.head()


,Mission
0,Apollo11
1,Apollo12
2,Apollo14
3,Apollo15
4,Apollo16


In [118]:
missions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Mission  6 non-null      object
dtypes: object(1)
memory usage: 176.0+ bytes


## Sum total weight by Mission
Adds a new column to the missions dataframe to represent the sum of all samples collected on that mission

In [119]:
sample_total_weight = rock_samples.groupby('Mission')['Weight(Kg)'].sum()
missions = pd.merge(missions, sample_total_weight, on='Mission')
missions.rename(columns={'Weight(Kg)' : 'Sample Weight(Kg)'}, inplace=True)
missions

,Mission,Sample Weight(Kg)
0,Apollo11,21.55424
1,Apollo12,34.34238
2,Apollo14,41.83363
3,Apollo15,75.39910
4,Apollo16,92.46262
5,Apollo17,109.44402


## Get the difference in weights across missions
Take a look at a lot of different cross sections of data that are available. The total weight of the samples increased with each mission, but it's hard to immediately see by how much.

Add one more column to the missions dataframe that simply grabs the difference between the current row and the row preceding it.

In [120]:
missions['Weight Diff'] = missions['Sample Weight(Kg)'].diff()
missions

,Mission,Sample Weight(Kg),Weight Diff
0,Apollo11,21.55424,NaN
1,Apollo12,34.34238,12.78814
2,Apollo14,41.83363,7.49125
3,Apollo15,75.39910,33.56547
4,Apollo16,92.46262,17.06352
5,Apollo17,109.44402,16.98140


## Fills null values with zero

In [121]:
missions['Weight Diff'] = missions['Weight Diff'].fillna(value=0)
missions

,Mission,Sample Weight(Kg),Weight Diff
0,Apollo11,21.55424,0.00000
1,Apollo12,34.34238,12.78814
2,Apollo14,41.83363,7.49125
3,Apollo15,75.39910,33.56547
4,Apollo16,92.46262,17.06352
5,Apollo17,109.44402,16.98140


## Add in command and lunar module data

By using the [NASA Space Science Data Coordinated Archive](https://nssdc.gsfc.nasa.gov/nmc/SpacecraftQuery.jsp), we gathered information about each module used in each mission. As you did when you created the samples tables, create six new columns, three for the lunar modules and three for the command modules:

* Module Name
* Module Mass
* Module Mass Diff

Fill in any NaN values with 0:

In [122]:
"""
    Lunar Modules (LM)
"""
missions['Lunar Module (LM)'] = {
    'Eagle (LM-5)',
    'Interpid (LM-6)',
    'Antares (LM-8)',
    'Falcon (LM-10)',
    'Orion (LM-11)',
    'Challenger (LM-12)'
}

missions['LM Mass (Kg)'] = {
    15103, 
    15235,
    15264,
    16430,
    16445,
    16456
}

missions['LM Mass Diff'] = missions['LM Mass (Kg)'].diff().fillna(value=0)

"""
    Command Modules (CM)
"""
missions['Command Module (CM)'] = {
    'Command (CSM-107)',
    'Yankee Clipper (CM-108)',
    'Kitty Hawk (CM-110)',
    'Endeavor (CM-112)',
    'Casper (CM-113)',
    'America (CM-114)'
}

missions['CM Mass (Kg)'] = {
    5560,
    5609,
    5758,
    5875,
    5840,
    5960
}

missions['CM Mass Diff'] = missions['CM Mass (Kg)'].diff().fillna(value=0)


missions

,Mission,Sample Weight(Kg),Weight Diff,Lunar Module (LM),LM Mass (Kg),LM Mass Diff,Command Module (CM),CM Mass (Kg),CM Mass Diff
0,Apollo11,21.55424,0.00000,Antares (LM-8),15264,0.0,Command (CSM-107),5960,0.0
1,Apollo12,34.34238,12.78814,Eagle (LM-5),15235,-29.0,Casper (CM-113),5609,-351.0
2,Apollo14,41.83363,7.49125,Interpid (LM-6),16456,1221.0,Yankee Clipper (CM-108),5840,231.0
3,Apollo15,75.39910,33.56547,Orion (LM-11),16430,-26.0,Endeavor (CM-112),5875,35.0
4,Apollo16,92.46262,17.06352,Falcon (LM-10),16445,15.0,Kitty Hawk (CM-110),5560,-315.0
5,Apollo17,109.44402,16.98140,Challenger (LM-12),15103,-1342.0,America (CM-114),5758,198.0


## Adding Total Weight Columns

In [123]:
missions['Total Weight (Kg)'] = missions['LM Mass (Kg)'] + missions['CM Mass (Kg)']
missions['Total Weight Diff'] = missions['LM Mass Diff'] + missions['CM Mass Diff']

missions

,Mission,Sample Weight(Kg),Weight Diff,Lunar Module (LM),LM Mass (Kg),LM Mass Diff,Command Module (CM),CM Mass (Kg),CM Mass Diff,Total Weight (Kg),Total Weight Diff
0,Apollo11,21.55424,0.00000,Antares (LM-8),15264,0.0,Command (CSM-107),5960,0.0,21224,0.0
1,Apollo12,34.34238,12.78814,Eagle (LM-5),15235,-29.0,Casper (CM-113),5609,-351.0,20844,-380.0
2,Apollo14,41.83363,7.49125,Interpid (LM-6),16456,1221.0,Yankee Clipper (CM-108),5840,231.0,22296,1452.0
3,Apollo15,75.39910,33.56547,Orion (LM-11),16430,-26.0,Endeavor (CM-112),5875,35.0,22305,9.0
4,Apollo16,92.46262,17.06352,Falcon (LM-10),16445,15.0,Kitty Hawk (CM-110),5560,-315.0,22005,-300.0
5,Apollo17,109.44402,16.98140,Challenger (LM-12),15103,-1342.0,America (CM-114),5758,198.0,20861,-1144.0


## Compare the Data
The interesting thing about predicting how much sample each Artemis mission can bring back is that we don't yet know the full specs of the spacecraft that the Artemis plans on using. Using some information from the [NASA Factsheet on the Space Launch System (SLS) and Orion Modules](https://www.nasa.gov/sites/default/files/atoms/files/0080_sls_fact_sheet_sept2020_09082020_final_0.pdf), we have have data on weights and payloads.

A payload is basically the total amount of weight that a rocket can get up through our atmosphere and into space. So the likelihood that the payload number is more accurate than the exact weights of each module is high, because deciding the payload will likely affect each of the other design decisions.

We know that the Saturn V payload was 43,500 kg, and the weights of the modules varied from mission to mission. So, to determine the ratios that will allow us to make predictions about the Artemis missions, we can use:

* Saturn V payload
* Mission sample weight
* Mission module weight


In [124]:
"""
    Sample to Weight Ratio
"""
saturnVPayload = 43500
missions['Crewed Area : Payload'] = missions['Total Weight (Kg)'] / saturnVPayload
missions['Sample : Crewed Area']  = missions['Sample Weight(Kg)'] / missions['Total Weight (Kg)']
missions['Sample : Payload']      = missions['Sample Weight(Kg)'] / saturnVPayload

missions

,Mission,Sample Weight(Kg),Weight Diff,Lunar Module (LM),LM Mass (Kg),LM Mass Diff,Command Module (CM),CM Mass (Kg),CM Mass Diff,Total Weight (Kg),Total Weight Diff,Crewed Area : Payload,Sample : Crewed Area,Sample : Payload
0,Apollo11,21.55424,0.00000,Antares (LM-8),15264,0.0,Command (CSM-107),5960,0.0,21224,0.0,0.487908,0.001016,0.000495
1,Apollo12,34.34238,12.78814,Eagle (LM-5),15235,-29.0,Casper (CM-113),5609,-351.0,20844,-380.0,0.479172,0.001648,0.000789
2,Apollo14,41.83363,7.49125,Interpid (LM-6),16456,1221.0,Yankee Clipper (CM-108),5840,231.0,22296,1452.0,0.512552,0.001876,0.000962
3,Apollo15,75.39910,33.56547,Orion (LM-11),16430,-26.0,Endeavor (CM-112),5875,35.0,22305,9.0,0.512759,0.003380,0.001733
4,Apollo16,92.46262,17.06352,Falcon (LM-10),16445,15.0,Kitty Hawk (CM-110),5560,-315.0,22005,-300.0,0.505862,0.004202,0.002126
5,Apollo17,109.44402,16.98140,Challenger (LM-12),15103,-1342.0,America (CM-114),5758,198.0,20861,-1144.0,0.479563,0.005246,0.002516
